Get Prediction Deliverables
--

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import math, sys
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
start_date = '2018-06-03'
end_date = '2018-10-13'
prediction_daily = './Clean_Daily_V311_APE_XGB_9_70_3_03_OUT'
prediction_hourly = './Clean_Daily2Hourly_V311_APE_XGB_9_70_3_03_OUT'

In [3]:
store_list = [7, 8, 9, 10, 14, 17, 18, 19, 25, 31, 32, 34, 35, 37, 41, 43, 48, 49, 54, 55, 57, 60, 62, 63, 64, 70, 71, 72, 73, 75, 77, 79, 84, 86, 89, 91, 95, 96, 97, 101, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 117, 118, 119, 121, 123, 125, 126, 127, 129, 130, 132, 134, 136, 138, 139, 140, 141, 143, 147, 148, 149, 153, 154, 155, 156, 158, 159, 160, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 191, 193, 195, 196, 197, 198, 202, 204, 205, 206, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 239, 240, 241, 242, 244, 247, 248, 249, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 262, 263, 264, 265, 266, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 280, 282, 283, 284, 285, 286, 287, 288, 289, 290, 292, 293, 296, 297, 298, 299, 301, 302, 303, 304, 305, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 367, 368, 369, 370, 371, 372, 373, 377, 379, 380, 381, 382, 383, 384, 385, 386, 388, 389, 394, 395, 397, 398, 399, 401, 403, 405, 406, 407, 408, 409, 411, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 425, 426, 427, 428, 429, 430, 431, 432, 433, 435, 436, 437, 438, 439, 440, 441, 442, 444, 445, 446, 447, 449, 450, 451, 453, 454, 455, 456, 457, 458, 460, 461, 462, 463, 464, 465, 466, 468, 469, 470, 471, 472, 473, 474, 475, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 493, 494, 495, 496, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 537, 538, 539, 540, 542, 544, 545, 547, 548, 549, 550, 551, 552, 553, 555, 557, 559, 560, 562, 563, 564, 565, 567, 568, 569, 570, 571, 574, 575, 576, 577, 579, 581, 584, 585, 586, 587, 588, 589, 590, 591, 593, 595, 596, 598, 601, 602, 605, 608, 609, 611, 612, 613, 614, 615, 616, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 630, 631, 633, 634, 636, 638, 639, 640, 642, 643, 644, 645, 646, 652, 653, 654, 656, 657, 658, 659, 662, 663, 664, 667, 668, 669, 673, 674, 675, 677, 678, 679, 680, 681, 682, 685, 686, 687, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 701, 702, 703, 704, 705, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 720, 721, 722, 723, 724, 725, 726, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 745, 746, 747, 748, 749, 752, 753, 755, 756, 757, 758, 759, 760, 761, 762, 763, 765, 766, 767, 768, 769, 770, 771, 773, 774, 776, 777, 778, 779, 780, 782, 783, 784, 785, 786, 787, 788, 790, 795, 796, 797, 798, 799, 951, 952, 953, 954, 955, 956, 957, 958, 960, 962, 963, 964, 967, 968, 969, 971, 973, 975, 976, 977, 980, 981, 982, 983, 985, 986, 987, 988, 990, 991, 992, 994, 995, 997, 998, 1002, 1003, 1004, 1006, 1007, 1008, 1009, 1010, 1011, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1027, 1028, 1030, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1040, 1041, 1042, 1043, 1045, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1055, 1056, 1057, 1058, 1059, 1060, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1075, 1076, 1077, 1078, 1079, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1089, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1115, 1117, 1118, 1119, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1145, 1147, 1149, 1151, 1153, 1155, 1156, 1157, 1158, 1159, 1160, 1161, 1162, 1163, 1165, 1166, 1167, 1169, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 1190, 1191, 1192, 1194, 1195, 1196, 1197, 1198, 1199, 1200, 1202, 1203, 1204, 1205, 1207, 1209, 1211, 1212, 1215, 1216, 1217, 1219, 1221, 1223, 1224, 1225, 1227, 1228, 1230, 1232, 1233, 1236, 1237, 1238, 1240, 1241, 1242, 1243, 1245, 1249, 1250, 1253, 1255, 1256, 1257, 1261, 1262, 1264, 1265, 1266, 1267, 1268, 1269, 1272, 1275, 1277, 1279, 1281, 1282, 1283, 1284, 1285, 1286, 1288, 1290, 1291, 1292, 1293, 1294, 1295, 1300, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1324, 1326, 1327, 1328, 1329, 1330, 1332, 1334, 1340, 1342, 1343, 1347, 1349, 1350, 1352, 1353, 1354, 1355, 1356, 1357, 1359, 1361, 1362, 1363, 1365, 1370, 1372, 1373, 1374, 1377, 1378, 1381, 1382, 1383, 1385, 1386, 1388, 1389, 1390, 1391, 1393, 1394, 1395, 1396, 1397, 1399, 1400, 1401, 1403, 1404, 1405, 1407, 1408, 1409, 1410, 1411, 1413, 1415, 1416, 1418, 1419, 1420, 1421, 1422, 1424, 1425, 1426, 1428, 1429, 1430, 1431, 1432, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1445, 1447, 1453, 1454, 1455, 1456, 1458, 1462, 1463, 1466, 1467, 1468, 1469, 1470, 1474, 1476, 1480, 1481, 1482, 1483, 1484, 1487, 1492, 1493, 1494, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1506, 1507, 1508, 1510, 1515, 1516, 1517, 1518, 1519, 1520, 1522, 1529, 1530]

In [4]:
# Get the first daily prediction.
predict_daily = pd.read_csv(prediction_daily + '/Error_APE_7.csv')
predict_daily.drop('Unnamed: 0', inplace=True, axis=1)
predict_daily['trn_sls_dte'] = pd.to_datetime(predict_daily['Date'], infer_datetime_format=True)
predict_daily['Date'] = predict_daily['trn_sls_dte'].dt.strftime('%Y-%m-%d')
predict_daily = predict_daily[(predict_daily['Date'] >= start_date) & (predict_daily['Date'] <= end_date)]
predict_daily = predict_daily.assign(STR_ID=[7 for i in range(len(predict_daily['Date']))])
predict_daily = predict_daily[['STR_ID', 'Date', 'VSTR_IN_CNT', 'VSTR_IN_CNT_pred']]


# Get the first hourly prediction.
predict_hourly = pd.read_csv(prediction_hourly + '/Error_APE_7.csv')
predict_hourly.drop('Unnamed: 0', inplace=True, axis=1)
predict_hourly['trn_sls_dte'] = pd.to_datetime(predict_hourly['Date'], infer_datetime_format=True)
predict_hourly['Date'] = predict_hourly['trn_sls_dte'].dt.strftime('%Y-%m-%d')
predict_hourly = predict_hourly[(predict_hourly['Date'] >= start_date) & (predict_hourly['Date'] <= end_date)]
predict_hourly = predict_hourly[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'pred_hourly']]

In [5]:
predict_daily

,STR_ID,Date,VSTR_IN_CNT,VSTR_IN_CNT_pred
121,7,2018-06-03,3508.0,3149.876709
122,7,2018-06-04,2236.0,2134.440674
123,7,2018-06-05,2201.0,2040.486206
124,7,2018-06-06,2267.0,2262.565186
125,7,2018-06-07,3317.0,2796.037598
126,7,2018-06-08,3651.0,3455.651123
127,7,2018-06-09,4649.0,4327.266113
128,7,2018-06-10,3213.0,3068.143555
129,7,2018-06-11,2688.0,2548.058105
130,7,2018-06-12,2702.0,2663.385254


In [6]:
predict_hourly.head(24)

,STR_ID,Date,SALEHOUR,IN_Clean,pred_hourly
10704,7,2018-06-03,0,0.0,0.000000
10705,7,2018-06-03,1,0.0,0.000000
10706,7,2018-06-03,2,0.0,0.000000
10707,7,2018-06-03,3,0.0,0.000000
10708,7,2018-06-03,4,0.0,0.000000
10709,7,2018-06-03,5,0.0,0.000000
10710,7,2018-06-03,6,0.0,0.000000
10711,7,2018-06-03,7,0.0,0.000000
10712,7,2018-06-03,8,0.0,0.000000
10713,7,2018-06-03,9,70.0,44.742567


In [7]:
for store_id in [i for i in store_list if i != 7]:
    # For each store.
    t_daily = pd.read_csv(prediction_daily + '/Error_APE_' + str(store_id) + '.csv')
    t_daily.drop('Unnamed: 0', inplace=True, axis=1)
    t_daily['trn_sls_dte'] = pd.to_datetime(t_daily['Date'], infer_datetime_format=True)
    t_daily['Date'] = t_daily['trn_sls_dte'].dt.strftime('%Y-%m-%d')
    t_daily = t_daily[(t_daily['Date'] >= start_date) & (t_daily['Date'] <= end_date)]
    t_daily = t_daily.assign(STR_ID=[store_id for i in range(len(t_daily['Date']))])
    t_daily = t_daily[['STR_ID', 'Date', 'VSTR_IN_CNT', 'VSTR_IN_CNT_pred']]
    
    # Union Result.
    predict_daily = pd.concat([predict_daily, t_daily])
    
    # Get hourly for each store
    t_hourly = pd.read_csv(prediction_hourly + '/Error_APE_' + str(store_id) + '.csv')
    t_hourly.drop('Unnamed: 0', inplace=True, axis=1)
    t_hourly['trn_sls_dte'] = pd.to_datetime(t_hourly['Date'], infer_datetime_format=True)
    t_hourly['Date'] = t_hourly['trn_sls_dte'].dt.strftime('%Y-%m-%d')
    t_hourly = t_hourly[(t_hourly['Date'] >= start_date) & (t_hourly['Date'] <= end_date)]
    t_hourly = t_hourly[['STR_ID', 'Date', 'SALEHOUR', 'IN_Clean', 'pred_hourly']]

    # Union hourly result.
    predict_hourly = pd.concat([predict_hourly, t_hourly])

In [9]:
predict_daily.to_csv('./000Deliverables/20181105_XGB_9_70_3_03_OUT/20181105_XGB_9_70_3_03_daily_OUT.csv', index=False)
predict_hourly.to_csv('./000Deliverables/20181105_XGB_9_70_3_03_OUT/20181105_XGB_9_70_3_03_hourly_OUT.csv', index=False)